# ORIE 4580: Intro Class Demos
#### *Sid Banerjee, Cornell*



Notebook with demos of Buffon's Needle, SIS epidemic, Wilson's algorithm and random linear maps

## Preamble

In [12]:
# Make sure we have the packages we need

import numpy as np
import scipy as sc
import math

# Configuring matplotlib
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.style.use('dark_background')
plt.rcParams["image.cmap"] = 'Set3'

# Choosing a colormap for the plot colors
cmap=plt.get_cmap('Set3')

# Buffon's needle simulation

First we demonstrate the Buffon's needle simulation - this is an example of using randomness for performing complex computations (in this case, computing $\pi$). We also introduce some basic animation techniques in python. 

See [Wikipedia article](https://en.wikipedia.org/wiki/Buffon%27s_needle_problem) for details of the problem.



Main simulation function, and generating the data

In [13]:
def buffon(N,length,dist,num_lines = 2,seed=-1):
    """
    Generates a design matrix with Gaussian basis functions
    
    Parameters
    ----------
    N: number of throws 
    length: length of toothpick
    dist: distance between lines
    num_lines: number of lines (should be >=2)
    seed: seed for random number generator; -1 for no seed
        
    Returns
    ----------
    sim_data: list of length N with outputs of sims
              each simulation output comprises:
              (tuples) (x_1,y_1), (x_2,y_2) of toothpick endpoints
              (int) number of line crossings
    
    """

    # Setting seed for PRNG (using -1 for no seed)
    if seed!=-1:
      np.random.seed(seed)              

    # Generate random centers and angles of toothpicks    
    # np.random.rand() generates a U[0,1] random variable
    xcent = 1+(num_lines-1)*np.random.rand(N)
    ycent = 1+(num_lines-1)*np.random.rand(N)
    theta = (np.pi/2.0)*np.random.rand(N)
    
    # Count number of times each toothpick touches a line
    crossed = (ycent - np.floor(ycent) - (length/2.0)*np.sin(theta) < 0).astype(int) + (ycent - np.floor(ycent) + (length/2.0)*np.sin(theta) > dist).astype(int)

    # Return toothpick endpoints, number of times each toothpick crosses    
    sim_data  = list(zip(list(zip(xcent - (length/2.0)*np.cos(theta),ycent - (length/2.0)*np.sin(theta))),
                     list(zip(xcent + (length/2.0)*np.cos(theta),ycent + (length/2.0)*np.sin(theta))),
                     crossed))

    return sim_data

In [14]:
# Generating the simulation data
N = 200
num_lines = 5
length = 1
dist = 1
data = buffon(N,length,dist,num_lines,seed=1)
crossed = np.cumsum([data[i][2] for i in range(N)])

## Static plots of simulation data

In [17]:
# For displaying static plots, matplotlib inline mode works better
%matplotlib inline

In [ ]:
from matplotlib.collections import LineCollection

lines = []
colors = []
for i in range(N):
  lines.append([(data[i][0][0],data[i][0][1]),(data[i][1][0],data[i][1][1])])
  colors.append(cmap(data[i][2]+4))
  
fig, ax = plt.subplots(figsize=(8,8))
ax.set(xlim=(0,num_lines+1), ylim=(0,num_lines+1))
ax.hlines(np.arange(1,num_lines+1), 0, num_lines+1, colors='white', linestyles='dashed')
ax.set_title('Buffon\'s needle simulation')

line_segments = LineCollection(lines,colors=colors)
ax.add_collection(line_segments)

plt.show()

In [ ]:
offset = 1   
n = np.arange(1,N+1)    
estim = np.zeros(N)
for i in n:
    estim[i-1] = 2.0*i/max(1,crossed[i-1])
    
fig, ax = plt.subplots(figsize=(16,8))
    
ax.plot(n[offset:],estim[offset:],'.',c=cmap(0))
ax.plot(n[offset:],np.pi*np.ones(len(n[offset:])),'--',c=cmap(3),lw=2)
ax.set_xlabel('Number of throws')
ax.set_ylabel('Estimate')
ax.set_title('Estimating $\pi$ via Buffon\'s Needle Experiment')
plt.show()

## Animated plots of simulation data



One way to generate matplotlib animations is using the FuncAnimation function (see [docs](https://matplotlib.org/3.3.1/api/animation_api.html)). 

This requires using matplotlib in the interactive mode (see [blog post](https://medium.com/@1522933668924/using-matplotlib-in-jupyter-notebooks-comparing-methods-and-some-tips-python-c38e85b40ba1))

In [20]:
# displaying animation works better with matplotlib in interactive mode
%matplotlib notebook

In [21]:
# Create animation using matplotlib.animation 

# The following command suppresses outputs of cell
# We use it here to avoid displaying blank figure
%%capture

from IPython.display import HTML
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots(figsize=(8,8))
ax.set(xlim=(0,num_lines+1), ylim=(0,num_lines+1))
ax.hlines(np.arange(1,num_lines+1), 0, num_lines+1, colors='green', linestyles='dashed')

disp_template = 'throws = %i\ncrossed = %i'
disp_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

def animate(i):
    line = ax.plot((data[i][0][0],data[i][1][0]),(data[i][0][1],data[i][1][1]),c=cmap(data[i][2]+4),lw=2)  
    disp_text.set_text(disp_template % (i+1,crossed[i]))
    return line, disp_text

# First way to display animation. Need to use 'jshtml' for Google Colab, 'html5' for localhost
plt.rc('animation', html='jshtml')
ani = FuncAnimation(fig, animate, N, interval=200, blit=True)

In [ ]:
# Now we can display the animation
ani

More complex animation with mupltiple subplots

In [10]:
%%capture
# If animation is too big, may need to increase embed limit
#plt.rcParams['animation.embed_limit'] = 2**128

cmap=plt.get_cmap('spring')

# create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))

ax1.set_title('Buffon\'s needle simulation')
ax1.set(xlim=(0,num_lines+1), ylim=(0,num_lines+1))
ax1.hlines(np.arange(1,num_lines+1), 0, num_lines+1, colors='white', linestyles='dashed')
ax1.axis('off')
ax1.set_facecolor('k')

n = np.arange(N)
cr_min = np.min([2.0*(n[i]+1)/max(1.0,crossed[i]) for i in range(N)])
cr_max = np.max([2.0*(n[i]+1)/max(1.0,crossed[i]) for i in range(N)])
ax2.set_title('Estimate for $\pi$')
ax2.set(xlim=(0,N+1), ylim=(cr_min,cr_max))
ax2.hlines(np.pi, 0, N+1, colors=cmap(0), linestyles='dashed')
ax2.set_xlabel('Number of throws')
ax2.set_ylabel('Estimate')

disp_template = 'throws = %i\ncrossed = %i'
disp_text = ax1.text(0.05, 0.9, '', transform=ax1.transAxes)

# intialize two line objects (one in each axes)
line1, = ax1.plot([], [], lw=2)
line2, = ax2.plot([], [], lw=2)
line = [line1, line2]

def animate(i):
    line[0] = ax1.plot((data[i][0][0],data[i][1][0]),(data[i][0][1],data[i][1][1]),c=cmap(data[i][2]/1.001))  
    line[1] = ax2.plot(i+1,2.0*(i+1)/max(1,crossed[i]),'.',c=cmap(0.99))
    disp_text.set_text(disp_template % (i+1,crossed[i]))
    return line, disp_text

ani = FuncAnimation(fig, animate, N, interval=100, blit=True)

In [ ]:
# Alternate way for displaying animation
HTML(ani.to_html5_video())

# The SIS epidemic

Next we demonstrate an example of a discrete event simulation model. 

Below, we simulate the [SIS epidemic model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIS_model) on a $2$-d grid

In [28]:
# We switch back to inline plots
%matplotlib inline

In [29]:
def update_infection(N,beta,kappa,I_t):
    """
    Given set of infected nodes I, update infection
    Parameters
    ----------
    N: (int) Size of grid (NxN)
    beta: infection rate (recovery rate set to 1)
    I_t: Set of infected neighbors at time t
    Returns
    ----------
    I_tplus: Set of infected neighbors at time t+1
    """
    I_tplus = I_t
    Lambda = 1.0 + 4.0*beta + kappa
    event = np.random.choice(['r','i','n'], p=[1.0/Lambda, 4.0*beta/Lambda,kappa/Lambda])
    if event == 'r':
      node_heal = tuple(np.random.randint(1,N+1,2))
      if node_heal in I_tplus: I_tplus.remove(node_heal)
    elif event == 'i':
      N_grid = np.arange(1,N+1)
      node_spread = tuple(np.random.randint(1,N+1,2))
      if node_spread in I_tplus:
        dirs = np.concatenate((np.eye(2),0-np.eye(2)),axis=0).astype(int)  
        node_infect = tuple(map(sum, zip(node_spread, dirs[np.random.choice(4)])))    
        if (node_infect[0] in N_grid) and (node_infect[1] in N_grid):
          if node_infect not in I_tplus: I_tplus.append(node_infect)
    else:
      node_infect = tuple(np.random.randint(1,N+1,2))
      I_tplus.append(node_infect)            
    return I_tplus        


def draw_network(N,I,fig):
  plt.xlim(0,N+1)
  plt.ylim=(0,N+1)
  n = np.arange(1,N+1)
  plt.plot(np.repeat(n,N),np.tile(n,N), '.g',markerfacecolor='None')
  for node in I:
    plt.plot(node[0],node[1], '.r')

We first generate simulation data for two different parameter settings (here we adjust the infection rate $\beta$ which corresponds to a net infection rate '$R0$')

In [32]:
N = 30
N_grid = np.arange(1,N+1)

np.random.seed(6)              
beta1 = 0.45
beta2 = 0.55
kappa = 1.0/(100.0*(N**2))
I1=[]
I2=[]
start = [(int(N/4),int(N/4)),(int(3*N/4),int(3*N/4))]
I1_t = [node for node in start]
I2_t = [node for node in start]
I1.append([node for node in I1_t])  
I2.append([node for node in I2_t])  

n_frames = 100
t_step = 1200

for n in range(n_frames):
  for t in range(t_step):
    I1_t = update_infection(N,beta1,kappa,I1_t)
    I2_t = update_infection(N,beta2,kappa,I2_t)
  I1.append([node for node in I1_t])  
  I2.append([node for node in I2_t])  

We next demonstrate an even easier way of animating a simulation, using Python's time.sleep() function

In [ ]:
from IPython import display
import time

for n in range(n_frames):
  display.clear_output(wait=True)
  fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
  fig.set_facecolor('k')
  fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
  
  ax1.set_title('SIS Epidemic: %ix%i grid, R0=%0.2f' % (N,N,2*beta1))
  ax2.set_title('SIS Epidemic: %ix%i grid, R0=%0.2f' % (N,N,2*beta2))
  ax1.set(xlim=(0,N+1), ylim=(0,N+1))
  ax2.set(xlim=(0,N+1), ylim=(0,N+1))
  ax1.axis('off')
  ax2.axis('off')
  ax1.plot(np.repeat(N_grid,N),np.tile(N_grid,N), 'og',markerfacecolor='None')
  ax2.plot(np.repeat(N_grid,N),np.tile(N_grid,N), 'og',markerfacecolor='None')
  for node in I1[n]:
    ax1.plot(node[0],node[1], 'or')
  for node in I2[n]:
    ax2.plot(node[0],node[1], 'or')
  plt.show()
  time.sleep(0)

# Random spanning trees

The earlier models involved very simple models and random variables. Later in the class, we will look at generating more complex random variables, for which an important technique is *Markov Chain Monte Carlo* (MCMC). We now see one such example.

Below, we demonstrate an MCMC technique for generating a *random spanning tree* in a graph (here, a 2-d grid). There are several techniques for this, but we demonstrate one of the most beautiful - [Wilson's algorithm](https://en.wikipedia.org/wiki/Loop-erased_random_walk#Uniform_spanning_tree).

In [34]:
# Choosing a colormap for the plot colors
cmap=plt.get_cmap('spring')

def random_nbr(N,node):
    """
    Given a node in NxN grid, returns a neighbor uniformly at random
    Parameters
    ----------
    N: (int) Size of grid (NxN)
    node: (tuple) node (x,y) 
    Returns
    ----------
    next: (tuple) Uniform random neigbor (x,y) of node 
    """
    N_grid = np.arange(1,N+1)
    dirs = np.concatenate((np.eye(2),0-np.eye(2)),axis=0).astype(int)  
    nbrs = [tuple(map(sum,zip(node,dirs[i]))) for i in range(len(dirs))]
    node_nbrs = []
    for new_node in nbrs:
        if (new_node[0] in N_grid) and (new_node[1] in N_grid):
            node_nbrs.append(new_node)
    return node_nbrs[np.random.choice(len(node_nbrs))]

def add_path(N,nodes_visited):
    """
    Adds a path to a tree via Wilson's algorithm
    
    Parameters
    ----------
    N: Size of grid (NxN)
    nodes_visited: list of nodes already added to tree
        
    Returns
    ----------
    
    """
    # Find smallest node not yet visited
    i = 0
    found = 0
    while not found:
      if (i%N+1,int(i/N)+1) not in nodes_visited:
          start = (i%N+1,int(i/N)+1)
          found = 1
      i = i+1    

    walk_collection = []
    path = [start]
    current_walk_start = start
    
    while path[-1] not in nodes_visited:
      next_node = random_nbr(N,path[-1])
      path.append(next_node)
      if next_node in path[:-1]:
        walk_collection.append([current_walk_start,path])
        ind = path.index(next_node)
        path = path[:ind+1]
        current_walk_start = path[-1]
      
    walk_collection.append([current_walk_start,path])
    return walk_collection,path


def draw_forest(N,tree,fig,shading=1):
  plt.xlim(0,N+1)
  plt.ylim=(0,N+1)
  n = np.arange(1,N+1)
  plt.plot(np.repeat(n,N),np.tile(n,N), ',w')

  count = 1.0
  for path in tree:
    last_node = path[0]
    plt.plot(last_node[0],last_node[1],'.g')
    for i in np.arange(1,len(path)):
      next_node = path[i]
      plt.plot(next_node[0],next_node[1],'.g')
      cmap1=plt.get_cmap('spring')
      if shading:
        plt.plot([last_node[0],next_node[0]],[last_node[1],next_node[1]], c=cmap1(count/len(tree)))
      else:
        plt.plot([last_node[0],next_node[0]],[last_node[1],next_node[1]], c=cmap1(0))  
      last_node = next_node
    count+=1.0     

In [ ]:
N = 30
start_node = (1,1)
np.random.seed(2)              

tree = []
paths_collection = []
nodes_visited = [start_node]

fig = plt.figure(figsize=(8,8))
plt.title('Wilson\'s algorithm: %ix%i grd' % (N,N))
plt.axis('off')
fig.set_facecolor(('xkcd:black'))

draw_forest(N,tree,fig)

while len(nodes_visited) < N*N:
  walks,path = add_path(N,nodes_visited)
  paths_collection = paths_collection + walks
  tree.append(path)
  for i in range(len(path)-1):
    nodes_visited.append(path[i])

draw_forest(N,tree,fig)
plt.show()

Again, we can do a simple animation of this algorithm

In [ ]:
N = 10
start_node = (1,1)
np.random.seed(2)              

tree = []
paths_collection = []
nodes_visited = [start_node]

fig = plt.figure(figsize=(8,8),facecolor='k')
plt.title('Wilson\'s algorithm: %ix%i grd' % (N,N))
plt.axis('off')
draw_forest(N,tree,fig)
plt.show()


while len(nodes_visited) < N*N:
  display.clear_output(wait=True)
  plt.figure(figsize=(8,8),facecolor='k')
  plt.title('Wilson\'s algorithm: %ix%i grd' % (N,N))
  plt.axis('off')
  walks,path = add_path(N,nodes_visited)
  paths_collection = paths_collection + walks
  tree.append(path)
  for i in range(len(path)-1):
    nodes_visited.append(path[i])
  draw_forest(N,tree,fig,shading=1)
  plt.show()
  time.sleep(0.001)

# Random Linear Maps

Finally, another important idea in this course is in understanding how random processes behave when run over long periods. We now look at one strange such process...

In [ ]:
# Generating the samples

# Number of samples
N = 4

# Initializing the random number generator seed
np.random.seed(0)
num_corners = 3
corners = np.array([[0,0],[1,2],[2,0]])

#num_corners = 5
#corners = np.array([[0.5,0],[0,2],[1.5,0],[2,2],[1,3]])

start_point = np.array([1,1])

# Initialize list of points
points = np.zeros((N+1,2))
points[0,:] = start_point

rand_choice = np.random.choice(num_corners,N)
    
for n in range(N):
    vertex_choice = corners[rand_choice[n]]
    points[n+1] = 0.5*(points[n])+ 0.5*(vertex_choice)
    
points_x = points[:,0]
points_y = points[:,1]    
    
fig = plt.figure(figsize=(10,10))
plt.axis('off')
fig.set_facecolor(('xkcd:black'))

from IPython import display
import time

for i in range(N):
  display.clear_output(wait=True)
  fig = plt.figure(figsize=(10,10))
  plt.axis('off')
  fig.set_facecolor(('xkcd:black'))
  plt.scatter(corners[:,0],corners[:,1],s=20, c='r', marker='o')        
  plt.scatter(points[:i,0],points[:i,1],s=20, c='b', marker='o')    
  plt.show()
  time.sleep(0.01)

In [ ]:
# Generating the samples

# Number of samples
N = 100000

# Initializing the random number generator seed
np.random.seed(0)
num_corners = 3
corners = np.array([[0,0],[1,2],[2,0]])

#num_corners = 5
#corners = np.array([[0.5,0],[0,2],[1.5,0],[2,2],[1,3]])

start_point = np.array([1,1])

# Initialize list of points
points = np.zeros((N+1,2))
points[0,:] = start_point

rand_choice = np.random.choice(num_corners,N)
    
for n in range(N):
    vertex_choice = corners[rand_choice[n]]
    points[n+1] = 0.5*(points[n])+ 0.5*(vertex_choice)
    
points_x = points[:,0]
points_y = points[:,1]    
    
fig = plt.figure(figsize=(10,10))
plt.axis('off')
fig.set_facecolor(('xkcd:black'))

plt.scatter(corners[:,0],corners[:,1],s=20, c='r', marker='o')        
plt.scatter(points[:,0],points[:,1],s=0.05, c='b', marker='.')    
plt.show()

In [ ]:
# Generating the samples

# Number of samples
N = 50000

# Initializing the random number generator seed
np.random.seed(0)

# Generate random variable array
rand_choice = np.random.choice(4, N, replace=True, p=[0.02, 0.84, 0.07, 0.07])



# Initialize list of points
points = np.zeros(shape = (N+1,2))

start_point = np.array([0,0])
points[0,:] = start_point


# Define linear operators for the Barnsley fern
A0 = np.matrix([[0,0],[0,0.16]])
A1 = np.matrix([[0.85,0.04],[-0.04,0.85]])
A2 = np.matrix([[0.2,-0.26],[0.23,0.22]])
A3 = np.matrix([[-0.15,0.28],[0.26,0.24]])
A = [A0,A1,A2,A3]
b = np.matrix([[0,0],[0,2.6],[0,2.6],[0,0.44]])

    
for n in range(N):
    points[n+1] = np.dot(A[rand_choice[n]],points[n].T)+b[rand_choice[n]] 

    
%matplotlib inline    
plt.figure(figsize = [10,10])
plt.scatter(points[:,0],points[:,1],s=10, c='g', marker='.')    
plt.show()